## Live acquisiton data management using - Pycroscopy Ecosystem:
- Converting to Sidpy datasets:
    - using utilities for sidpy datasets like - fitters, fft etc
- Using scifireaders to read them

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 9th May 2025
#### edited - 

### 0. Setup the live acquisiton

In [ ]:
from stemOrchestrator.logging_config import setup_logging

data_folder = "."
out_path = data_folder
setup_logging(out_path=out_path)

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import (
    HAADF_tiff_to_png,
    tiff_to_png,
    tiff_metadata,
    tiff_to_numpy,
)
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging

plot = plt
from typing import Dict

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")

if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")

try:
    port = int(port)
except ValueError:
    raise ValueError("Port must be an integer")

config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512,  # square
    "out_path": ".",
}

In [ ]:
ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]


microscope = TemMicroscopeClient()
microscope.connect(ip, port=port)  # 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])

### 1. Acquire an image

In [ ]:
image_with_metadata, image_array = tf_acquisition.acquire_haadf(
    resolution=512, exposure=0.00001, dont_save_but_return_object=True
)

### 2. convert it to sidpy dataset

In [ ]:
import sidpy
import numpy as np
import xml.etree.ElementTree as ET
from stemOrchestrator.process import etree_to_dict
from sidpy.sid import Dimension

In [ ]:
xml_str = image_with_metadata.metadata.metadata_as_xml
root = ET.fromstring(xml_str)
metadata_dict = etree_to_dict(root)

In [ ]:
image = image_array
data_sets = {"Channel_000": sidpy.Dataset.from_array(image.T, name="HAADF")}
data_sets["Channel_000"].data_type = "image"

ds = data_sets["Channel_000"]

ds.original_metadata = metadata_dict["Metadata"]

pixel_size_x_m = float(ds.original_metadata["BinaryResult"]["PixelSize"]["X"]["#text"])
pixel_size_y_m = float(ds.original_metadata["BinaryResult"]["PixelSize"]["Y"]["#text"])
pixel_size_x_um = pixel_size_x_m * 1e6
pixel_size_y_um = pixel_size_y_m * 1e6

ds.set_dimension(
    0,
    Dimension(
        np.arange(image.shape[0]) * pixel_size_y_um,
        name="y",
        units="µm",
        quantity="Length",
        dimension_type="spatial",
    ),
)

ds.set_dimension(
    1,
    Dimension(
        np.arange(image.shape[1]) * pixel_size_x_um,
        name="x",
        units="µm",
        quantity="Length",
        dimension_type="spatial",
    ),
)

# ds.metadata['note'] = 'The shape of overview image and spectrum image are different in 1st 2 dimensions, possibly due to binning.'

#### 2.a) use functionalities from sidpy datasets

In [ ]:
view = ds.plot()